## References

- https://asthasr.github.io/posts/how-blockchains-work/?utm_source=tldrnewsletter

## Get Started

In [1]:
# !/usr/bin/env python3
from argparse import ArgumentParser
from hashlib import md5


def hash_string(string):
    hash = md5()
    hash.update(string.encode("utf-8"))
    return hash.hexdigest()


# if __name__ == "__main__":
#     parser = ArgumentParser()
#     parser.add_argument("STRING", help="The string to be hashed")
#     args = parser.parse_args()
#     print(hash_string(args.STRING))

In [2]:
hash_string("ninja")

'3899dcbab79f92af727c2190bbd8abc5'

In [3]:
hash_string("samurai")

'99b1983cf3ee09bbaf6f43ac7b4c8748'

In [5]:
def hash_ledger_entry(string, previous_digest=None):
    """Hashes a string with the hash of previous entries in the ledger, if any."""
    hash = md5(string.encode("utf-8"))

    if previous_digest:
        hash.update(previous_digest.encode("utf-8"))

    return hash.hexdigest()


def generate_ledger(*strings):
    """Generates the entries in a ledger consisting of a set of strings."""
    digest = None

    for string in strings:
        digest = hash_ledger_entry(string, digest)
        yield digest, string


# if __name__ == "__main__":
#     parser = ArgumentParser()
#     parser.add_argument("STRINGS", help="The ledger entries", nargs="+")
#     args = parser.parse_args()

#     for digest, string in generate_ledger(*args.STRINGS):
#         print(f"{digest}\t{string}")

In [18]:
for digest, string in generate_ledger('ninja','pirate','samurai'):
    print(f"{digest}\t{string}")

3899dcbab79f92af727c2190bbd8abc5	ninja
7ec21dcf528e12036b04774754ecc4e0	pirate
636730d86709d03fed9ba64f84fc9be6	samurai


## Validation

In [19]:
# our_digest = None

# for line in fileinput.input():
#     file_digest, word = line.strip().split("\t")
#     our_digest = hash_ledger_entry(word, our_digest)

#     if our_digest != file_digest:
#         sys.exit(f"The digest for {word} does not match.")

# print("All entries match.")

## Proofs without Authority


In [22]:
# Add this to your imports.
from secrets import token_bytes


def hash_ledger_entry_with_salt(salt, string, previous_digest=None):
    """Hashes a string with the hash of previous entries in the ledger, if any."""
    hash = md5(string.encode("utf-8"))
    hash.update(salt)

    if previous_digest:
        hash.update(previous_digest.encode("utf-8"))

    return hash.hexdigest()


def generate_ledger(difficulty, *strings):
    # Difficulty determines how many zeroes we require at the beginning of a digest.
    prefix = "0" * difficulty

    digest = None
    previous_digest = None

    for string in strings:
        # We re-hash a string over and over, with random salts, until it matches the
        # prefix determined by our difficulty.
        while digest is None or not digest.startswith(prefix):
            salt = token_bytes(16)
            digest = hash_ledger_entry_with_salt(salt, string, previous_digest)

        # We yield back the digest and entry, as before, but we need the salt, too.
        # Without that, we can't replay the entries and verify them.
        yield digest, salt.hex(), string
        previous_digest = digest
        digest = None

    yield hash_ledger_entry_with_salt(salt, "END", previous_digest), salt, "END"


# if __name__ == "__main__":
#     parser = ArgumentParser()
#     parser.add_argument(
#         "DIFFICULTY", help="The difficulty of confirming a ledger entry.", type=int
#     )
#     parser.add_argument("STRINGS", help="The ledger entries", nargs="+")
#     args = parser.parse_args()

#     for digest, salt, string in generate_ledger(args.DIFFICULTY, *args.STRINGS):
#         print(f"{digest}\t{salt}\t{string}")

In [27]:
    for digest, salt, string in generate_ledger(5, 'ninja','pirate','samurai'):
        print(f"{digest}\t{salt}\t{string}")

00000814d6a2709e1b7866d62508b8ce	c045825a82536279cfa35e2ddcae1460	ninja
000007c8b60a88d3efad6d9032b9e506	76a7c4f3c3693092f2390ae62b5add84	pirate
00000672eb4785fc6219672dad254711	73336fd6777129841cd042c458426d63	samurai
8a55ee00350d7fa511cf3e13b1c2f34c	b's3o\xd6wq)\x84\x1c\xd0B\xc4XBmc'	END
